<font size="5">**Разложение Холецкого:**</font>

$A=LL^T$

$A$ - симметричная, положительно-определенная, $L$ - нижняя треугольная матрица со строго положительными элементами на диагонали

Алгоритм:

$l_{11}=\sqrt{a_{11}}$, $l_{j1}=\frac{a_{j1}}{l_{11}}$

$l_{ii}=\sqrt{a_{ii}-\sum_{p=1}^{i-1}l_{ip}^2}$

$l_{ji}=\frac{1}{l_{ii}}(a_{ji}-\sum_{p=1}^{i-1}l_{ip}l_{jp})$

In [105]:
import numpy as np
import scipy.linalg as LA
import datetime
np.set_printoptions(precision=2, linewidth = 200)

N=10

L0=np.random.randint(1,100,size=(N,N))
L0=np.tril(L0)

A = L0 @ L0.transpose()
x0 = np.random.randint(-10,10,size=(N,))
b = A @ x0

Решить систему:

$Ax=b$

Вариант 1 - использовать обратную матрицу:

$x=A^{-1}b$

In [106]:
t0=datetime.datetime.now()
M = 10000

for ind in np.arange(M):
    x = LA.inv(A) @ b

delta = (datetime.datetime.now() - t0).total_seconds()
print('dt={0:.2f} microseconds'.format(1e6*delta/M))

dt=63.36 microseconds


Вариант 2 - действовать напрямую:

(на самом деле, данный метод использует LU разложение)


In [107]:
t0=datetime.datetime.now()
M = 10000

for ind in np.arange(M):
    x=np.linalg.solve(A,b)
    
delta = (datetime.datetime.now() - t0).total_seconds()
print('dt={0:.2f} microseconds'.format(1e6*delta/M))

dt=15.69 microseconds


<font size="3">**LU - алгоритм**</font>

In [108]:
def diy_lu(a):
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [109]:
t0=datetime.datetime.now()
M = 1000

for ind in np.arange(M):
    L, U = diy_lu(A)
    y=LA.solve_triangular(L,b,lower=True,check_finite=False)
    x=LA.solve_triangular(U,y,lower=False,check_finite=False)
    
    
delta = (datetime.datetime.now() - t0).total_seconds()
print('dt={0:.2f} microseconds'.format(1e6*delta/M))

dt=199.90 microseconds


<font size="3">**Разложение Холецкого**</font>

In [110]:
def simple_cholesky(A):
    N=np.shape(A)[0]
    L=np.zeros_like(A)
    L[0,0]=np.sqrt(A[0,0])
    for j in np.arange(1,N):
        L[j,0]=A[j,0]/L[0,0]

    for i in np.arange(1,N):
        tmp_sum=0
        for k in np.arange(0,i):
            tmp_sum+=L[i,k]*L[i,k]

        L[i,i] = np.sqrt(A[i,i] - tmp_sum)
        if i<N-1:
            for j in np.arange(i+1,N):
                tmp_sum=0
                for k in np.arange(0,i):
                    tmp_sum+=L[i,k]*L[j,k]

                L[j,i] = 1.0 / L[i,i] * (A[j,i] - tmp_sum)
    return L

In [111]:
t0=datetime.datetime.now()
M = 1000

for ind in np.arange(M):
    L=simple_cholesky(A)
    y=LA.solve_triangular(L,b,lower=True,check_finite=False)
    x=LA.solve_triangular(L.T,y,lower=False,check_finite=False)
    
    
delta = (datetime.datetime.now() - t0).total_seconds()
print('dt={0:.2f} microseconds'.format(1e6*delta/M))

dt=638.63 microseconds


In [112]:
np.sort(x0)

array([-6, -4, -4, -3, -1, -1,  0,  1,  1,  6])

In [113]:
np.sort(x)

array([-6., -4., -4., -3., -1., -1.,  0.,  1.,  1.,  6.])

<font size="5">**Еще одно применение - генерация скореллированных случайных величин**</font>

Пусть $\Sigma=LL^T$ - матрица ковариации случайных величин, $X$ - вектор из независимых нормально распределенных случайных величин.

Тогда вектор $Y=LX$ будет иметь многомерное нормальное распределение с нулевым средним и заданной ковариационной матрицей $\Sigma$.


In [116]:
Nt=40000000 #num of points
Nvals=5 #num of vals

tmpM=np.random.randint(1,10,size=[Nvals,Nvals]) #generate covariance matrix
tmpM=np.tril(tmpM) #make lower triangular matrix

Sigma=tmpM.dot(tmpM.T) #inverse cholesky
L=np.linalg.cholesky(Sigma) #L==tmpM

In [117]:
X=np.random.multivariate_normal(np.zeros(Nvals),np.eye(Nvals),size=Nt).T #non correlated
Y=L @ X #correlated

Ковариационная матрица, рассчитанная по сгенерированным (некоррелированным) данным:

In [118]:
print(np.cov(X))

[[ 1.00e+00  8.63e-05 -7.44e-05 -1.35e-04 -5.78e-05]
 [ 8.63e-05  1.00e+00  1.94e-04  2.06e-05 -1.50e-04]
 [-7.44e-05  1.94e-04  1.00e+00 -1.53e-04  1.56e-04]
 [-1.35e-04  2.06e-05 -1.53e-04  1.00e+00 -2.20e-04]
 [-5.78e-05 -1.50e-04  1.56e-04 -2.20e-04  1.00e+00]]


Ковариационная матрица, рассчитанная по сгенерированным (коррелированным) данным:

In [119]:
print(np.cov(Y))

[[ 49.02  35.02  35.02  56.01  42.01]
 [ 35.02  61.02  79.03  52.03  48.01]
 [ 35.02  79.03 107.04  67.03  58.01]
 [ 56.01  52.03  67.03 173.97  92.99]
 [ 42.01  48.01  58.01  92.99 130.96]]


Заданная квариационная матрица:

In [120]:
print(Sigma)

[[ 49  35  35  56  42]
 [ 35  61  79  52  48]
 [ 35  79 107  67  58]
 [ 56  52  67 174  93]
 [ 42  48  58  93 131]]


Относительная разница по норме:

In [121]:
dN=np.sqrt(np.sum((Sigma-np.cov(Y))**2))
N=np.sqrt(np.sum(Sigma**2))
print(dN/N)

0.00028295488706551574


<font size="5">**Спойлер**</font>

Зачем нужны скоррелированные многомерные случайные величины?

Например, рассмотрим уравнение диффузии:

<font size="4">
$\frac{\partial f(\boldsymbol{x},t)}{\partial t}=-\sum_i\frac{\partial}{\partial x_i}[\mu_i (\boldsymbol{x},t)f(\boldsymbol{x},t)]+\sum_i\sum_j\frac{\partial^2}{\partial x_i \partial x_j}[D_{ij}(\boldsymbol{x},t))f(\boldsymbol{x},t)]$
</font>

Это уравнение можно решать несколькими способами:

1. напрямую (численно, используя разностные схемы)
2. используя "тестовые частицы"
3. прибегая к использованию стохастических дифференциальных уравнений:

<font size="4">
$d\boldsymbol{x_t}=\mu_i(\boldsymbol{x_t},t)dt+\sigma(\boldsymbol{x_t},t)d\boldsymbol{W_t}$
</font>

где $\boldsymbol{W_t}$ -  многомерный винеровский процесс, а тензор $D=\frac{1}{2}\sigma\sigma^T$